In [1]:
import csv
import os
import numpy as np
import librosa
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
import tensorflow as tf
import matplotlib.pyplot as plt
from pywt import wavedec

2023-10-10 22:03:12.624135: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-10 22:03:12.674611: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-10 22:03:12.674646: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-10 22:03:12.674670: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-10 22:03:12.681748: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-10 22:03:12.682184: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [4]:
# import os
# dataset_path = '../Preprocessing/db6'
# for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
#     for f in filenames:
#         file_path = os.path.join(dirpath, f)
#         print(file_path)


In [5]:
# header = 'filename'
# header += ' hasil'
# header = header.split()

# file = open('../Preprocessing/hasil.csv', 'w', newline='')
# with file:
#   writer = csv.writer(file)
#   writer.writerow(header)

In [4]:
import os
dataset_path = '../Preprocessing/'
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
    for d in dirnames:
      for j, (dirpath2, dirnames2, filenames2) in enumerate(os.walk(os.path.join(dirpath, d))):
        for f in filenames2:
          file_path = os.path.join(dirpath2, f)
          print("Processing file: ", file_path)
          df = pd.read_csv(file_path)

          X = df.drop(['label', 'filename'], axis=1)
          y = df.iloc[:, -1]

          X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

          lb = LabelEncoder()
          X_train = np.expand_dims(X_train, axis=2)
          X_test = np.expand_dims(X_test, axis=2)
          y_test = tf.keras.utils.to_categorical(lb.fit_transform(y_test))
          y_train = tf.keras.utils.to_categorical(lb.fit_transform(y_train))

          def build_model():
            model = models.Sequential(
                [
                    # layers.Conv1D(64, (3), activation="relu", input_shape=(X_train.shape[1], X_train.shape[2])),
                    # layers.MaxPooling1D((3), strides=(2), padding="same"),

                    # layers.Conv1D(32, (3) , activation="relu"),
                    # layers.MaxPooling1D((3), strides=(2), padding="same"),

                    # layers.Flatten(),
                    # layers.Dense(32, activation="relu"),
                    # layers.Dense(5, activation="softmax"),

                    layers.Conv1D(64, (3), activation="relu", input_shape=(X_train.shape[1], X_train.shape[2])),
                    layers.MaxPooling1D((3), strides=(2), padding="same"),

                    layers.Conv1D(32, (3) , activation="relu"),
                    layers.MaxPooling1D((3), strides=(2), padding="same"),

                    layers.Flatten(),
                    layers.Dense(64, activation="relu", activity_regularizer=tf.keras.regularizers.l2(1e-4)),
                    layers.Dense(5, activation="softmax"),
                ]
            )

            return model

          model = build_model()


          optimazer = tf.keras.optimizers.Adam(learning_rate=0.0001)
          model.compile(optimizer=optimazer, loss='categorical_crossentropy', metrics=['acc'])
          # history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))
          history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))
          test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

          to_append = f'{file_path}'
          to_append += f' {test_accuracy}'
          file = open('../Preprocessing/hasil.csv', 'a', newline='')
          with file:
              writer = csv.writer(file)
              writer.writerow(to_append.split())

Processing file:  ../Preprocessing/db1/data_MfccDwtRmsCqtdb1L5.csv
Epoch 1/50
45/45 [==============================] - 1s 10ms/step - loss: 1.8447 - acc: 0.2417 - val_loss: 1.6071 - val_acc: 0.4000
Epoch 2/50
45/45 [==============================] - 0s 5ms/step - loss: 1.3591 - acc: 0.5361 - val_loss: 1.2532 - val_acc: 0.5889
Epoch 3/50
45/45 [==============================] - 0s 4ms/step - loss: 1.0913 - acc: 0.6653 - val_loss: 0.9919 - val_acc: 0.6889
Epoch 4/50
45/45 [==============================] - 0s 4ms/step - loss: 0.8957 - acc: 0.7486 - val_loss: 0.8260 - val_acc: 0.7611
Epoch 5/50
45/45 [==============================] - 0s 4ms/step - loss: 0.7668 - acc: 0.7889 - val_loss: 0.7112 - val_acc: 0.7889
Epoch 6/50
45/45 [==============================] - 0s 4ms/step - loss: 0.6813 - acc: 0.8000 - val_loss: 0.6405 - val_acc: 0.8389
Epoch 7/50
45/45 [==============================] - 0s 4ms/step - loss: 0.6236 - acc: 0.8208 - val_loss: 0.5846 - val_acc: 0.8500
Epoch 8/50
45/45 [====

In [7]:
# dataset_path = '../Preprocessing/db10'
# for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
#   for f in filenames:
#     file_path = os.path.join(dirpath, f)
#     print('Processing: ', file_path)
#     data = pd.read_csv(file_path)
#     # data.head()

#     X = data.drop(['label', 'filename'], axis=1)
#     y = data.iloc[:, -1]

#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     lb = LabelEncoder()
#     X_train = np.expand_dims(X_train, axis=2)
#     X_test = np.expand_dims(X_test, axis=2)
#     y_test = tf.keras.utils.to_categorical(lb.fit_transform(y_test))
#     y_train = tf.keras.utils.to_categorical(lb.fit_transform(y_train))

#     X_train = X_train[..., np.newaxis]
#     X_test = X_test[..., np.newaxis]

#     def build_model():
#         model = models.Sequential(
#             [
#                 layers.Conv1D(64, (3), activation="relu", input_shape=(X_train.shape[1], X_train.shape[2])),
#                 layers.MaxPooling1D((3), strides=(2), padding="same"),

#                 layers.Conv1D(32, (3) , activation="relu"),
#                 layers.MaxPooling1D((3), strides=(2), padding="same"),

#                 layers.Flatten(),
#                 layers.Dense(64, activation="relu"),
#                 layers.Dense(5, activation="softmax"),
#             ]
#         )

#         return model

#     model = build_model()


#     optimazer = tf.keras.optimizers.Adam(learning_rate=0.0001)
#     model.compile(optimizer=optimazer, loss='categorical_crossentropy', metrics=['acc'])
#     history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


#     # evaluate model on test set
#     test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
#     print("Accuracy on test set is: {}".format(test_accuracy))

#     to_append = f'{file_path}'
#     to_append += f' {test_accuracy}'
#     file = open('../Preprocessing/hasil.csv', 'a', newline='')
#     with file:
#         writer = csv.writer(file)
#         writer.writerow(to_append.split())
